In [1]:
# !pip install alibi
# !pip install tensorflow

In [2]:
%load_ext autoreload

import sys
sys.path.insert(0, "../src")

import os
os.chdir("../src")

In [3]:
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from data_generator import DataGenerator
from counterfactual import Counterfactual
from utils import lift_analysis
import tensorflow as tf
from tensorflow.python.framework.ops import disable_eager_execution
np.random.seed(0)
tf.random.set_seed(0)
disable_eager_execution()

In [4]:
data_handler = DataGenerator("../data/imputed_bank_data_mice.csv",
                             imputed=True, preprocessed=False)
x_train, x_test, y_train, y_test = data_handler.load_data()


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [5]:
# load the retrained model
with open('../results/Models/LGBM/model.pkl', 'rb') as file:
    lgbm_model = joblib.load(file)

# define a CFP explainer object
cfp_explainer = Counterfactual(lgbm_model, x_train, data_handler)

No encoder specified. Using k-d trees to represent class prototypes.


In [6]:
# select an instance you want to study
instance = x_test.loc[32171]

# calculate the counterfactuals if any.
counterfactual = cfp_explainer.define_counterfactuals(instance)

# convert to original form and display
instance_orig = data_handler.rev_transform(instance)
print("--------------------")
print("\nOriginal Instance:")
display(instance_orig)

if counterfactual is not None:
  counterfactual_orig = data_handler.rev_transform(counterfactual.iloc[0])
  print("\nCounterfactual Instance:")
  display(counterfactual_orig)

Original instance: no  -- proba: [0.99336939 0.00663061]
Counterfactual instance: yes  -- proba: [0.34770054 0.65229946]

Counterfactual perturbations...

Categorical:
day_of_week: fri  -->   thu
binned_duration: 200sec - 400sec  -->   more than 800sec

Numerical:
--------------------

Original Instance:


,education,job,marital,month,day_of_week,binned_duration,poutcome,housing,loan,contact,age,campaign,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,basic,blue-collar,married,may,fri,200sec - 400sec,nonexistent,no,no,cellular,50.0,6.0,0.0,-2.0,93.0,-46.0,1.0,5099.0



Counterfactual Instance:


,education,job,marital,month,day_of_week,binned_duration,poutcome,housing,loan,contact,age,campaign,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,basic,blue-collar,married,may,thu,more than 800sec,nonexistent,no,no,cellular,50.0,6.0,5.548320e-09,-2.0,93.0,-46.0,1.0,5099.0


In [7]:
# select an instance you want to study
instance = x_test.loc[2831]

# calculate the counterfactuals if any.
counterfactual = cfp_explainer.define_counterfactuals(instance)

# convert to original form and display
instance_orig = data_handler.rev_transform(instance)
print("--------------------")
print("\nOriginal Instance:")
display(instance_orig)

if counterfactual is not None:
  counterfactual_orig = data_handler.rev_transform(counterfactual.iloc[0])
  print("\nCounterfactual Instance:")
  display(counterfactual_orig)

No counterfactual found!


--------------------

Original Instance:


,education,job,marital,month,day_of_week,binned_duration,poutcome,housing,loan,contact,age,campaign,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,professional course,technician,married,may,wed,less than 200sec,nonexistent,yes,no,telephone,50.0,2.0,0.0,1.0,94.0,-36.0,5.0,5191.0


In [8]:
# select an instance you want to study
x = ["housemaid", "divorced", "high school",  "no", "no", "cellular", "may", "tue",
     "nonexistent", "200sec - 400sec", 40, 2.0, 0.0, -3.0, 93.0, -30.0, 1.0, 5018.0]
instance = data_handler.transform(x)

# calculate the counterfactuals if any.
counterfactual = cfp_explainer.define_counterfactuals(instance)

# convert to original form and display
instance_orig = data_handler.rev_transform(instance)
print("--------------------")
print("\nOriginal Instance:")
display(instance_orig)

if counterfactual is not None:
  counterfactual_orig = data_handler.rev_transform(counterfactual.iloc[0])
  print("\nCounterfactual Instance:")
  display(counterfactual_orig)

Original instance: no  -- proba: [0.91879231 0.08120769]
Counterfactual instance: yes  -- proba: [0.36955663 0.63044337]

Counterfactual perturbations...

Categorical:
binned_duration: 200sec - 400sec  -->   400sec - 800sec

Numerical:
--------------------

Original Instance:


,education,job,marital,month,day_of_week,binned_duration,poutcome,housing,loan,contact,age,campaign,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,high school,housemaid,divorced,may,tue,200sec - 400sec,nonexistent,no,no,cellular,40.0,2.0,0.0,-3.0,93.0,-30.0,1.0,5018.0



Counterfactual Instance:


,education,job,marital,month,day_of_week,binned_duration,poutcome,housing,loan,contact,age,campaign,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,high school,housemaid,divorced,may,tue,400sec - 800sec,nonexistent,no,no,cellular,40.0,2.0,5.548320e-09,-3.0,93.0,-30.0,1.0,5018.0


## Instance:

In [9]:
'''
illiterate, unemployed ~ 35
uni_deg, unemployed ~ 35

technician, married, may, tue, 200-400sec ~ 30 

highschool, housemaid, divroced, may, tue, 200-400sec  ~ 40
'''

'\nilliterate, unemployed ~ 35\nuni_deg, unemployed ~ 35\n\ntechnician, married, may, tue, 200-400sec ~ 30 \n\nhighschool, housemaid, divroced, may, tue, 200-400sec  ~ 40\n'

In [10]:
# select an instance you want to study
x = ["unemployed", "unmarried", "illiterate",  "no", "no", "cellular", "may", "tue",
     "nonexistent", "200sec - 400sec", 35, 2.0, 0.0, -3.0, 93.0, -30.0, 1.0, 5018.0]
instance = data_handler.transform(x)

# calculate the counterfactuals if any.
counterfactual = cfp_explainer.define_counterfactuals(instance)

# convert to original form and display
instance_orig = data_handler.rev_transform(instance)
print("--------------------")
print("\nOriginal Instance:")
display(instance_orig)

if counterfactual is not None:
  counterfactual_orig = data_handler.rev_transform(counterfactual.iloc[0])
  print("\nCounterfactual Instance:")
  display(counterfactual_orig)

ValueError: Number of features of the model must match the input. Model n_features_ is 55 and input n_features is 56 